# 搭建中文分词工具

In [1]:
from collections import Counter
import random
import numpy as np
with open('../data/cidian.txt','r',encoding='utf-8') as f:
    lines = f.readlines()

sum = 0
# Counter 是实现的 dict 的一个子类，可以用来方便地计数
word_prob = Counter()

for line in lines:
    columns = line.strip().split()
#     print(columns)
    # 重复词频率直接相加，（相同词多次出现是因为发音不同，即语义也不同，这里不做区分）
    word_prob[columns[0]] += int(columns[-1])
    sum += int(columns[-1])
#     strip() 方法用于移除字符串头尾指定的字符
# str2 = "   Runoob      ";   # 去除首尾空格
# print str2.strip();
# split() 通过指定分隔符对字符串进行切片
# print str.split( );       # 以空格为分隔符，包含 \n
# 频率转为概率
for word in word_prob:
    word_prob[word] /= sum
print([{word: word_prob[word]} for word in random.sample(word_prob.keys(), 3)])
print("词典大小：%d" % len(word_prob))
print(np.sum(list(word_prob.values())))
# print(word_prob)

[{'辞令': 2.1071789762535106e-05}, {'暮景': 3.334498645933082e-05}, {'太师椅': 1.750484420946574e-05}]
词典大小：55735
1.0


In [15]:
def sentence_cut(str):
#     思路：状态转移，设M[i]是从句子开始到第i个字所组成句的全切分，word是以字i结尾的可在词典中找到的词，
#     则M[i] = M[i-len(word)] + word
    memory = [[] for _ in range(len(str))]
#     ’_’ 是一个循环标志，也可以用i，j 等其他字母代替，下面的循环中不会用到，起到的是循环此数的作用
#     列表解析，链接：https://blog.csdn.net/qq_27361945/article/details/79816843
    for i in range(0, len(str)):
        for j in range(0, i+1):
            # 从开始到当前index视为一个词
            if j == 0:
                if str[j:i+1] in word_prob:
                    memory[i].append([str[j:i+1]])
                continue
            # 确定依赖的之前状态存在且（达成转移条件：词存在）
            if memory[j-1] and str[j:i+1] in word_prob:
                for state in memory[j-1]:
#                     print(state)
#                     print('\n')
#                     print(str[j:i+1])
                    memory[i].append(state+[str[j:i+1]])
#     print(memory)
    return memory[-1]
print(sentence_cut("北京的天气真好啊"))

[['北京', '的', '天气', '真', '好', '啊'], ['北', '京', '的', '天气', '真', '好', '啊'], ['北京', '的', '天', '气', '真', '好', '啊'], ['北', '京', '的', '天', '气', '真', '好', '啊']]


循环所有的分词结果，并计算出概率最高的分词结果，并返回

In [30]:
import math 
def word_segment(str):
    # 计算所有可能的分词结果，要保证每个分完的词存在于词典里,调用上面的函数
    segments = sentence_cut(str)
    best_score = 0
    best_segment = list()
    for seg in segments:
        if seg:
            score = 0
            for word in seg:
                #使用log是为了防止太多0.00000000~1导致下溢
                score += math.log(word_prob[word])
            if best_score == 0:
                best_score = score
                best_segment = seg
            else:
                if score > best_score:
                    best_score = score
                    best_segment = seg
    return best_segment

def sentence_cut(str):
    memory = [[] for _ in range(len(str))]
    for i in range(0, len(str)):
        for j in range(0, i+1):
            if j == 0:
                if str[j:i+1] in word_prob:
                    memory[i].append([str[j:i+1]])
                continue
            if memory[j-1] and str[j:i+1] in word_prob:
                for state in memory[j-1]:
                    memory[i].append(state+[str[j:i+1]])
    return memory[-1]

print(word_segment("经常有意见分歧"))

['经常', '有意', '见', '分歧']


# 维特比算法
原方法将分词和计算概率分开，这里使用状态转移的方法将二者同时进行

In [31]:
import math
def word_segment_viterbi(str):
    # 第一步：根据词典，输入的句子，以及给定的unigram概率来创建带权重的有向图（Directed Graph）
    #      有向图的每一条边是一个单词的概率（只要存在于词典里的都可以作为一个合法的单词），这些概率在 word_prob，如果不在word_prob里的单词但在
    #      词典里存在的，统一用概率值1e-100。
    # 图是为了直观起见，边表示字或词及其概率，节点存储状态，图有没有其实无所谓，从本质上讲其实就是个状态转移算法
    # 每个节点的状态包含-log(P)和当前最优切分
    memory = [[0,[]] for _ in range(len(str)+1)]
     # 第二步： 利用维特比算法来找出最好的PATH， 这个PATH是P(sentence)最大或者 -log P(sentence)最小的PATH。
    # 第三步： 根据最好的PATH, 返回最好的切分
    for i in range(1, len(str)+1):
        for j in range(i):
            word = str[j:i]
            #查询本地词典里这个词的概率
#             prob = word_prob[word] if word in word_prob else 1e-100
            if word in word_prob:
                prob = word_prob[word]
            else: 
                prob = 1e-100
            score = memory[j][0] - math.log(prob)
            # 状态更新
            if memory[i][0] == 0:
                memory[i][0] = score
                memory[i][1] = memory[j][1] + [word]
            else:
                if score < memory[i][0]:
                    memory[i][0] = score
                    memory[i][1] = memory[j][1] + [word]
#     print(memory)
    return memory[-1][-1]
segments = word_segment_viterbi("经常有意见分歧")
print(segments)

['经常', '有意', '见', '分歧']


链接：https://zhuanlan.zhihu.com/p/95599399